In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline



In [ ]:
df=pd.read_csv("../input/titanic/train.csv",index_col="PassengerId")



# Data preprocessing.
1.Cleaning

In [ ]:
l,b=df.shape
df["Age"].fillna(df["Age"].median(),inplace=True)
df["Cabin"].fillna("ab",inplace=True)
df["Embarked"].fillna("S",inplace=True)


In [ ]:
#df["Sex"]=df["Sex"].replace({"male":str(0),"female":str(1)})
#filt=(df["Survived"]==1)
#dff=df[["Survived","Sex"]]
#dff=dff.groupby("Sex")
#dff["Survived"].sum().plot(kind="bar")

In [ ]:
df["Sex"]=df["Sex"].replace({"male":str(1),"female":str(0)}).astype(int)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["Embarked"]=le.fit_transform(df["Embarked"])

removing outliers.

In [ ]:
q=df["Fare"].quantile(0.99)
df2=df[df["Fare"]<q]
tar=df2['Survived']
fea=df2.drop(["Survived","Name","Ticket","Cabin"],axis=1)



In [ ]:
#scaler=MinMaxScaler()
#fea[["Fare","Age"]]=scaler.fit_transform(fea[["Fare","Age"]])

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(fea,tar,test_size=0.3,random_state=42,shuffle=True)

In [ ]:
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.metrics import confusion_matrix , accuracy_score
from xgboost import XGBClassifier
reg=XGBClassifier()


In [ ]:
params={"n_estimators":[67,70,100,120],'reg_lambda':[2,1],'gamma':[0,0.3,0.2,0.1]
       ,'eta':[0.06,0.05,0.04]
        ,"max_depth":[3,5],'objective':['binary:logistic']}



In [ ]:
clf=GridSearchCV(reg,params,cv=10,n_jobs=-1,verbose=1)
clf.fit(xtrain,ytrain)

In [ ]:
ypred=clf.predict(xtrain)
tpred=clf.predict(xtest)
print(accuracy_score(ypred,ytrain))
print(accuracy_score(tpred,ytest))
clf.best_params_

In [ ]:
#gen=pd.read_csv("C:/Users/hp/Downloads/gender_submission (1).csv")
#gpred=gen["Survived"]

In [ ]:
tes=pd.read_csv("../input/titanic/test.csv",index_col="PassengerId")


In [ ]:
tes["Age"]=tes["Age"].fillna(tes["Age"].median())
tes["Sex"]=le.fit_transform(tes["Sex"])
tes["Embarked"]=le.fit_transform(tes["Embarked"])
tes["Fare"]=tes["Fare"].fillna(tes["Fare"].median())
tes.isnull().sum()
tfea=tes.drop(["Name","Ticket","Cabin"],axis=1)
#tfea[["Fare","Age"]]=scaler.fit_transform(tfea[["Fare","Age"]])
my_pred=clf.predict(tfea)
submit=pd.DataFrame(tes.index,columns=["PassengerId"])
submit["Survived"]=my_pred
submit